In [65]:
from pokemon_types import gen6
from functools import lru_cache
from itertools import combinations, chain

How many types are there, what are they?

In [21]:
print(sorted(gen6))
len(gen6)

['bug', 'dark', 'dragon', 'electric', 'fairy', 'fighting', 'fire', 'flying', 'ghost', 'grass', 'ground', 'ice', 'normal', 'poison', 'psychic', 'rock', 'steel', 'water']


18

In [39]:
@lru_cache(maxsize=3*len(gen6))
def effective(type_, eff):
    return {k for k, v in gen6[type_].items() if v == eff}

In [16]:
print(effective('bug', 2.0))
print(effective('bug', 0.5))

{'dark', 'grass', 'psychic'}
{'steel', 'fairy', 'poison', 'fire', 'ghost', 'fighting', 'flying'}


What's the minimum sized subset of types that will be super effective to all other types?

In [40]:
def touches(types, eff):
    return set.union(*(effective(t, eff) for t in types))

def cover(k):
    all_types = set(gen6)
    sols = []
    for combo in combinations(all_types, k):
        if touches(combo, 2.0) == all_types:
            sols.append(combo)
    return sols

In [81]:
print(cover(6))
sorted(sorted(types) for types in cover(7))

[]


[['dark', 'electric', 'fighting', 'flying', 'ground', 'ice', 'poison'],
 ['dark', 'electric', 'fighting', 'flying', 'ground', 'ice', 'steel'],
 ['dark', 'fairy', 'fighting', 'grass', 'ground', 'poison', 'rock'],
 ['dark', 'fighting', 'flying', 'grass', 'ground', 'ice', 'poison'],
 ['dark', 'fighting', 'flying', 'grass', 'ground', 'ice', 'steel'],
 ['electric', 'fighting', 'flying', 'ghost', 'ground', 'ice', 'poison'],
 ['electric', 'fighting', 'flying', 'ghost', 'ground', 'ice', 'steel'],
 ['fairy', 'fighting', 'ghost', 'grass', 'ground', 'poison', 'rock'],
 ['fighting', 'flying', 'ghost', 'grass', 'ground', 'ice', 'poison'],
 ['fighting', 'flying', 'ghost', 'grass', 'ground', 'ice', 'steel']]

That's cool, because in standard Pokémon games, a team can consist of up to 6 members.

We can go further by asking for a team that also has minimum weaknesses. Let's arbitrarily define a score.

In [64]:
def score(types):
    a = touches(types, 2.0)
    b = touches(types, 0.5)
    c = touches(types, 0.0)
    return 2*len(a) - len(b) - 2*len(c)

In [80]:
combos = ((score(combo), combo, len(combo)) for combo
           in chain.from_iterable(combinations(gen6, k) for k in range(1, 7)))
sorted(combos, reverse=True)[:10]

[(18, ('ice', 'steel', 'ground', 'ghost', 'fairy', 'flying'), 6),
 (18, ('ice', 'steel', 'dark', 'ground', 'fairy', 'rock'), 6),
 (18, ('ice', 'steel', 'dark', 'ground', 'fairy', 'flying'), 6),
 (18, ('dark', 'ground', 'grass', 'bug', 'fairy', 'rock'), 6),
 (17, ('water', 'steel', 'ground', 'fire', 'fairy', 'electric'), 6),
 (17, ('water', 'steel', 'ground', 'fairy', 'flying', 'electric'), 6),
 (17, ('water', 'steel', 'ghost', 'fire', 'fairy', 'electric'), 6),
 (17, ('water', 'steel', 'dark', 'fire', 'fairy', 'electric'), 6),
 (17, ('steel', 'ground', 'grass', 'bug', 'fairy', 'rock'), 6),
 (17, ('steel', 'ground', 'ghost', 'grass', 'fire', 'fairy'), 6)]